# **Bloom-560m**

In [1]:
!pip install transformers torch textstat language-tool-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.4 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Initialize the FALCON model and tokenizer
model_name = "bigscience/bloom-560m"  # FALCON model for instruction-based generation
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate articles using FALCON with improved sampling techniques
def generate_article(prompt, max_length=500, temperature=0.7, top_p=0.9, repetition_penalty=1.2):
    inputs = tokenizer(prompt, return_tensors="pt")
    attention_mask = torch.ones_like(inputs['input_ids'])  # Set attention mask
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=temperature,  # Adjusts creativity
        top_p=top_p,  # Top-p sampling for more diversity
        repetition_penalty=repetition_penalty,  # Penalizes repetition
        num_return_sequences=3
    )
    article = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return article

# Example usage
prompt = "How does climate change affect agriculture in various regions around the world?"
article = generate_article(prompt)

print("Generated Article:\n")
print(article)

import language_tool_python
import textstat

# Initialize LanguageTool for grammar checking
tool = language_tool_python.LanguageTool('en-US')

# Performance Metric 1: Grammar Check
def grammar_check(article):
    matches = tool.check(article)
    errors = len(matches)
    return errors

# Performance Metric 2: Readability Score
def readability_score(article):
    score = textstat.flesch_reading_ease(article)
    return score

# Performance Metric 3: Word Count
def word_count(article):
    words = article.split()
    return len(words)

# Performance Metric 4: Perplexity for FALCON
def calculate_perplexity(article):
    inputs = tokenizer(article, return_tensors="pt")
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    perplexity = torch.exp(torch.tensor(loss))
    return perplexity.item()

# Evaluate all metrics
def evaluate_article(article):
    errors = grammar_check(article)
    readability = readability_score(article)
    word_count_value = word_count(article)
    perplexity_value = calculate_perplexity(article)

    print(f"Performance Metrics:\n")
    print(f"Grammar Errors: {errors}")
    print(f"Readability Score: {readability}")
    print(f"Word Count: {word_count_value}")
    print(f"Perplexity: {perplexity_value}")

# Example usage
evaluate_article(article)

Generated Article:

How does climate change affect agriculture in various regions around the world? In this study, we examine changes to plant communities across an agroecosystem and find that most species of crop plants are not affected by climatic variability. Furthermore our results suggest a direct link between agricultural productivity (crop production) under increasing temperatures as well increase precipitation with increased temperature.
This article is based on work published online at https://journals-plant-soil-technology-associates-publishing-online-reviewer .org
MATERIALS AND METHODS

Study area: China Agricultural Research Institute's Experimental Station for Plant Bioecology & Biology System 1 -China Agriculture University Extension Service Field sampling sites: Four fields were selected from three different locations within China's Northeast region during July 2014 through October 2015; each field was sampled once per year until May 2016 using two plots located approxim